## Segementation of cells

Currently I have only been testing with simulation output from LBIBCell (rendered by Paraview).

Will walk through the basic workflow below

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from numpy import mean
import pandas as pd
import numpy as np
from scipy import ndimage as ndi
from scipy.stats import variation, gstd
from skimage import (
    color, filters, measure, morphology, segmentation, io, feature
)

## IO: Reading screenshot from tiff or png files

The external saved tiff, png or other image format

Now only works with one frame (a single image)

Can change it to gray scale for now

In [2]:
# set parameters related to simulation
size_x, size_y = 1000, 1000

filename = 'seg_lbibcell_example.png'

In [3]:
# read img as greyscale
img = io.imread(filename, as_gray=True)
print('Input: {} X {}'.format(img.shape[0], img.shape[1]))

# scale factor size the simulation takes place in a fixed box grid
SCALE_X = img.shape[0] / size_x
SCALE_Y = img.shape[1] / size_y
AREA_SCALE = SCALE_X * SCALE_Y
SMALL_OBJECTS_THRESHOLD = 40

Input: 4000 X 4000


## Segmentation methods

For now, since the photos are simulated, no denoising or other restoration are made.

Simple threshold filter would be good results provided the input image is fine enought (high resolution needed)

1. [Segment human cells (in mitosis)](https://scikit-image.org/docs/dev/auto_examples/applications/plot_human_mitosis.html#sphx-glr-auto-examples-applications-plot-human-mitosis-py)
2. [Other techniques by scikit-image, and evaluation](https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_metrics.html#sphx-glr-auto-examples-segmentation-plot-metrics-py)

The true result should be 3120 cells

### Edge-based segmentation (canny)

In [4]:
def print_label_info(labels, img, AREA_SCALE):
    region_props = ['label', 'area', 'centroid']
    props = measure.regionprops_table(labels, img,properties=region_props)
    print('{} cells identified from the image'.format(labels.max()))
    print('Average area : {:.4f} +/- {:.2f}'.format(mean(props['area']) / AREA_SCALE, gstd(props['area']) / AREA_SCALE))
    print('CV           : {:.4f}'.format(variation(props['area'])))
    print('min          : {:.4f}'.format(np.min(props['area']) / AREA_SCALE))
    print('max          : {:.4f}'.format(np.max(props['area']) / AREA_SCALE))
    
    return props

In [ ]:
img_edge_canny = feature.canny(img)

cells_fill_canny = ndi.binary_fill_holes(img_edge_canny)

cells_fill_canny_clean = morphology.remove_small_objects(cells_fill_canny, SMALL_OBJECTS_THRESHOLD * AREA_SCALE)

labeled_cells_canny = measure.label(cells_fill_canny_clean)

props = print_label_info(labeled_cells_canny, img, AREA_SCALE)

### Region-based segmentation

In [ ]:
img_sobel = filters.sobel(img)

markers = np.zeros_like(img)
markers[img < 0.25] = 0
markers[img > 0.25] = 1

cells_sobel_seg = segmentation.watershed(img_sobel, markers)

### Filtering: Multi-Otsu Thresholding

In [ ]:
# Use Multi-Otsu Thresholding
thresholds = filters.threshold_multiotsu(img, classes=2)

# get label cells, background is darker
# use simple filter since it
cells = img <= thresholds

# remove objects < 40, have to remove small artifacts
cells_clean = morphology.remove_small_objects(
    cells, SMALL_OBJECTS_THRESHOLD * AREA_SCALE)

labeled_cells_threshold_multiotsu = measure.label(cells_clean)

props = print_label_info(labeled_cells_canny, img, AREA_SCALE)

## Visualization (Only filter)

In [ ]:
fig, ax = plt.subplots()
plt.imshow(io.imread(filename))
ax.set_title('Original Image')
ax.set_axis_off()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(segmentation.mark_boundaries(img, labeled_cells_threshold_multiotsu))
ax.set_title('Multi-Otsu Thresholding (< {} removed)'.format(SMALL_OBJECTS_THRESHOLD))
ax.set_axis_off()

In [ ]:
fig, axes = plt.subplots()
plt.scatter(x=props['centroid-0'], y=props['centroid-1'], s=props['area'] / (img.shape[0] * 20))
ax.set_title('Centroid of cells')
ax.set_axis_off()

In [ ]:
image_label_overlay = color.label2rgb(labeled_cells_threshold_multiotsu, image=img, bg_label=0)
image_label_overlay = color.label2rgb(labeled_cells_threshold_multiotsu, image=img, bg_label=0)
fig, ax = plt.subplots()
plt.imshow(image_label_overlay)
ax.set_title('Labelled cells (Color)')
ax.set_axis_off()

Useful link:

1. https://scipy-lectures.org/packages/scikit-image/auto_examples/plot_boundaries.html#sphx-glr-packages-scikit-image-auto-examples-plot-boundaries-py
2. https://github.com/mrgloom/awesome-semantic-segmentation
3. https://emmanuelle.github.io/a-tutorial-on-segmentation.html a little outdated
4. https://scikit-image.org/docs/stable/auto_examples/applications/plot_coins_segmentation.html

And mostly tutorials on scikit-image